In [1]:
from IPython.display import Image

# 16. 순환 신경망으로 순차 데이터 모델링
* 순차 데이터
* 시퀀스 모델링을 위한 RNN
* 장단기 기억(Long Short-Term Memory, LSTM) 네트워크
* T-BPTT(Truncated BackPropagation Through Time)
* 시퀀스 모델링을 위한 다층 RNN 구현
* IMDb 영화 리뷰 데이터셋을 사용한 감정 분석
* 소설 텍스트 기반 LSTM 셀을 사용하여 RNN 글자 단위 언어 모델링
* 그레이디언트 클리핑(clipping)을 사용하여 그레이디언트 폭주 피하기
* 트랜스포머 모델 - 셀프 어텐션 매커니즘

# 16.1 순차 데이터

### 16.1.1 순차 데이터 모델링: 순서를 고려
* 일반적으로 지도 학습을 위한 머신러닝 알고리즘은 입력 데이터가 독립 동일 분포(Independent and Indentically Distributed, IID)라고 가정.
    * 훈련 샘플이 상호 독립적이고 같은 분포에 속한다는 의미 -> 순서와 관계가 없다.
* 시퀀스 원소들은 특성 순서가 있으므로 상호 독립적이지 않다.
    * 특정 주식의 가격을 예측
        * 각 훈련 샘플은 특정 날의 주식 가격을 나타냄
    

* 순차 데이터 vs 시계열 데이터
    * 시계열 데이터: 순차 데이터의 한 종류
        * 각 샘플이 시간 차원에 연관
        * 연속적인 타임스탬프에 따라 샘플을 얻음
            * 시간 차원이 데이터 포인트 사이의 순서를 결정
        * ex) 주식 가격, 녹화된 음성 또는 대화
    * 모든 순차 데이터가 시간 차원을 가지는 것은 아니다.
        * 텍스트 데이터나 DNA 시퀀스는 샘플이 순서를 가지지만 시계열 데이터는 아니다.

### 16.1.2 시퀀스 표현


In [2]:
# 시계열 데이터
Image(url='https://git.io/JLdVm', width=700)

입력 특성(x)와 타깃 레이블(y)은 시간 축을 따라 순서대로 나열. 따라서 x와 y는 시퀀스 데이터이다.

* RNN은 시퀀스 모델링을 위해 고안되었으며, 과거 정보를 기억하고 이에 맞추어 새로운 샘플을 처리할 수 있기 때문에 시퀀스 데이터를 다룰 때 장점을 가진다.

### 16.1.3 시퀀스 모델링 종류
* 시퀀스 모델링
    * ex) 언어 번역, 이미지 캡셔닝, 텍스트 생성

In [3]:
# 시퀀스 모델링의 종류
Image(url='https://git.io/JLdVO', width=700)

* 다대일(many-to-one)
    * 입력 데이터가 시퀀스이지만, 출력 데이터가 시퀀스가 아니고 고정 크기의 벡터나 스칼라이다.
    * ex) 감정 분석에서 입력은 텍스트(리뷰 등)이고, 출력은 클래스 레이블(긍부정 레이블 등).
* 일대다(one-to-many)
    * 입력 데이터가 시퀀스가 아니라 일반적인 형태이고, 출력은 시퀀스이다.
    * ex) 이미지 캡셔닝 - 입력이 이미지, 출력은 이미지 내용을 요약한 영어 문장
* 다대다(many-to-many)
    * 입력과 출력 배열이 모두 시퀀스
        * 이런 종류는 입력과 출력이 동기적인지에 따라 더 나눌 수 있다.
        * 동기적인 다대다 모델링 작업
            * ex) 각 프레임을 레이블링하는 비디오 분류
        * 지연이 있는 다대다 모델링 작업
            * ex) 한 언어에서 다른 언어로 번역하는 작업 - 독일어에서 영어로 번역할 때 전체 영어 문장을 읽어 처리한다.

# 16.2 시퀀스 모델링을 위한 RNN

### 16.2.1 RNN 반복 구조


In [4]:
# 피드포워드 신경망과 RNN
Image(url='https://git.io/JLdV3', width=700)

두 네트워크 모두 하나의 은닉층만 있다. 입력층(x), 은닉층(h), 출력층(o) 모두 벡터이고 여러 개의 유닛이 있다고 가정.

기본 피드포워드 네트워크에서 정보는 입력에서 은닉층으로 흐른 후 은닉층에서 출력층으로 전달도니다. 반면 순환 네트워크에서는 은닉층이 현재 타임 스텝의 입력층과 이전 타임 스텝의 은닉층으로부터 정보를 받는다.

인접한 타임 스텝의 정보가 은닉층에 흐르기 때문에 네트워크가 이전 이벤트를 기억할 수 있다. 이 정보 흐름을 보통 루프로 표시한다. 그래프 표기법에서는 순환 에지(recurrent edge)라고도 하기 때문에 RNN 구조 이름이 여기서 유래되었다.

다층 퍼셉트론과 비슷하게 RNN은 여러 개의 은닉층으로 구성할 수 있다. 하나의 은닉층을 가진 RNN을 관례적으로 단일층 RNN이라고 한다. 

* RNN 출력 형태 결정
    * 일반적인 RNN 구조는 시퀀스 모델링 중 입력이 시퀀스인 두 경우에 해당된다.
    * 전형적으로 순환 층은 시퀀스 출력 <o^(0), o^(1), ..., o^(T)>를 반환 하거나 마지막 출력(t=T일 때, 즉 o^(T))을 반환한다.
        * 다대다 또는 마지막 원소 o^(T)만 최종 출력으로 사용할 경우 다대일이 될 수 있다.
    * 텐서플로 케라스 API에서 순환 층이 시퀀스를 출력할지 마지막 출력만 반환할지 설정하기 위해 `return_sequences` 매개변수를 True 또는 False로 지정.


In [5]:
# 하나의 은닉층을 가진 RNN과 여러 개의 은닉층을 가진 RNN
Image(url='https://git.io/JLdVs', width=700)

RNN의 은닉 유닛은 두 개의 다른 입력을 받는다. 입력층으로부터 받은 입력과 같은 은닉층에서 t-1 타임 스텝의 활성화 출력을 받는다.

t=0에서는 은닉 유닛이 0 또는 작은 난수로 초기화된다. t > 0인 타임 스텝에서는 은닉 유닛이 현재 타임 스텝의 데이터 포인트 $x^t$와 이전 타임 스텝 t-1의 은닉 유닛 값 $h^{(t-1)}$을 입력으로 받는다

* 다층 RNN 정보 흐름
    * layer1: 은닉층의 출력을 $h_1^{(t)}$로 표현. 데이터 포인트 $x^{(t)}$와 이 은닉층의 이전 타임 스텝 출력 $h_1^{(t-1)}$을 입력으로 받는다.
    * layer2: 두 번째 은닉층의 $h_2^{(t)}$는 이전 층의 현재 타임 스텝 출력($o_1^{(t)}$)와 이 은닉층의 이전 타임 스텝 출력 $h_2^{(t-1)}$을 입력으로 받는다.

이 경우 각 은닉층은 시퀀스를 입력으로 받기 때문에 마지막을 제외하고 모든 순환 층은 시퀀스를 출력으로 반환해야 한다.(즉, return_sequences=True) 마지막 순환 층의 동작은 문제 유형에 따라 결정된다.

### 16.2.2 RNN의 활성화 출력 계산

* 단일층 RNN의 각 가중치
    * $W_{xh}$: 입력 $x^{(t)}$와 은닉층 h 사이의 가중치 행렬
    * $W_{hh}$: 순환 에지에 연관된 가중치 행렬
    * $W_{ho}$: 은닉층과 출력층 사이의 가중치 행렬

구현에 따라 가중치 행렬 $W_{xh}$와 $W_{hh}$를 합쳐 연결된 행렬 $W_h$=$[W_{wh};W_{hh}]$를 사용.

In [6]:
Image(url='https://git.io/JLdVC', width=700)

활성화 출력의 계산은 기본적인 다층 퍼셉트론이나 다른 피드포워드 신경망과 매우 비슷하다. 은닉층의 최종 입력 $Z_h$(활성화 함수를 통과하기 전의 값)는 선형 조합으로 계산한다. 즉, 가중치 행렬과 대응되는 벡터를 곱해서 더한 후 절편 유닛을 더한다. 그 다음 타임 스텝 t에서 은닉층의 활성화를 계산한다.

$$h^{(t)}=\phi_h(z_h^{(t)}) = \phi_h(W_{xh}x^{(t)}+W_{hh}h^{(t-1)}+b_h)$$

* $b_h$: 은닉 유닛의 절편 벡터
* $\phi_h()$: 은닉층의 활성화 함수

가중치 행렬 $W_h=[W_{xh};W_{hh}]$처럼 연결.

$$h^{(t)}=\phi([W_{xh};W_{hh})] \begin{bmatrix} x^{(t)}\\h^{(t-1)} \end{bmatrix}+b_h)$$

현재 타임 스텝에서 은닉 유닛의 활성화 출력을 계산한 후 출력 유닛의 활성화를 아래와 같이 계산한다.

$$o^{(t)}=\phi_o(W_{ho}h^{(t)}+b_o)$$



In [7]:
# 순환 신경망의 계산
Image(url='https://git.io/JLdVW', width=700)

### 16.2.3 은닉 순환과 출력 순환

출력층에서 오는 순환 연결을 가진 모델의 경우 이전 타임 스텝의 출력층에서 오는 활성화 $o^{(t-1)}$을 추가하는 방법
* 현재 타임 스텝에서 은닉층 $h^t$에 추가한다
* 현재 타임 스텝에서 출력층 $o^t$에 추가한다


In [8]:
# 순환 방식에 따른 RNN
# 1. 은닉-은닉 순환 W_hh
# 2. 출력-은닉 순환 W_oh
# 3. 출력-출력 순환 W_oo
Image(url='https://git.io/JLdV8', width=700)

이 순환 타입 중 하나의 정방향 계산을 수동으로 수행.

텐서플로 케라스 API의 SimpleRNN 클래스로 출력-출력 순환과 비슷한 순환 층을 정의할 수 있다. SimpleRNN으로 순환 층을 만들고 길이가 3인 입력 시퀀스에서 정방향 계산을 수행하여 출력을 만든다. 또한, 수동으로 정방향 계산을 수행하여 SimpleRNN의 결과와 비교해본다.

먼저 층을 만들고 수동 계산을 위해 가중치를 저장.

In [9]:
import tensorflow as tf

In [10]:
tf.random.set_seed(1)

rnn_layer = tf.keras.layers.SimpleRNN(
    units=2, use_bias=True,
    return_sequences=True)
rnn_layer.build(input_shape=(None, None, 5))

w_xh, w_oo, b_h = rnn_layer.weights

print('W_xh 크기:', w_xh.shape)
print('W_oo 크기:', w_oo.shape)
print('b_h 크기:', b_h.shape)

W_xh 크기: (5, 2)
W_oo 크기: (2, 2)
b_h 크기: (2,)


* 입력 크기는 (None, None, 5)
    * 첫 번째 차원은 배치 차원(가변적인 배치 크기를 위해 None으로 지정)
    * 두 번째 차원은 시퀀스에 해당(가변적인 시퀀스 길이를 위해 Noen으로 지정)
    * 마지막 차원은 특성에 해당

`return_sequences=True`로 지정했으므로 길이가 3인 시퀀스를 입력하면 출력 시퀀스 $<o^{(0)}, o^{(1)}, o^{(2)}>$가 나온다. 그렇지 않으면 최종 출력 $o^{(2)}$만 반환된다.

rnn_layer의 정방향 계산을 수행하고 각 타임 스텝에서 수동으로 출력을 계산하여 비교해본다.

In [11]:
x_seq = tf.convert_to_tensor(
    [[1.0]*5, [2.0]*5, [3.0]*5],
    dtype=tf.float32)


## SimepleRNN의 출력:
output = rnn_layer(tf.reshape(x_seq, shape=(1, 3, 5)))

## 수동으로 출력 계산하기:
out_man = []
for t in range(len(x_seq)):
    xt = tf.reshape(x_seq[t], (1, 5))
    print('타임 스텝 {} =>'.format(t))
    print('   입력           :', xt.numpy())
    
    ht = tf.matmul(xt, w_xh) + b_h    
    print('   은닉           :', ht.numpy())
    
    if t>0:
        prev_o = out_man[t-1]
    else:
        prev_o = tf.zeros(shape=(ht.shape))
        
    ot = ht + tf.matmul(prev_o, w_oo)
    ot = tf.math.tanh(ot)
    out_man.append(ot)
    print('   출력 (수동)     :', ot.numpy())
    print('   SimpleRNN 출력 :'.format(t), output[0][t].numpy())
    print()

타임 스텝 0 =>
   입력           : [[1. 1. 1. 1. 1.]]
   은닉           : [[0.41464037 0.96012145]]
   출력 (수동)     : [[0.39240566 0.74433106]]
   SimpleRNN 출력 : [0.39240566 0.74433106]

타임 스텝 1 =>
   입력           : [[2. 2. 2. 2. 2.]]
   은닉           : [[0.82928073 1.9202429 ]]
   출력 (수동)     : [[0.80116504 0.99129474]]
   SimpleRNN 출력 : [0.80116504 0.99129474]

타임 스텝 2 =>
   입력           : [[3. 3. 3. 3. 3.]]
   은닉           : [[1.243921  2.8803642]]
   출력 (수동)     : [[0.95468265 0.99930704]]
   SimpleRNN 출력 : [0.95468265 0.99930704]



* 수동으로 정방향 계산을 할 때 하이퍼볼릭 탄젠트(tanh) 활성화 함수를 사용
    * SimpleRNN에서 이 함수를 사용하기 때문 - 활성화 함수 기본값
    

### 16.2.4 긴 시퀀스 학습의 어려움

BPTT(BackPropagation Through Time)는 새로운 문제를 야기시킨다. 손실 함수의 그레이디언트를 계산할 때 곱셈 항때문에 소위 그레이디언트 폭주 또는 그레이디언트 소실 문제가 발생한다.

In [12]:
# 그레이디언트 소실과 폭주
Image(url='https://git.io/JLdV4', width=700)

가중치 w가 t-k번 곱해져 $w^{t-k}$가 된다. 결국 |w| < 1이면 t-k가 클 때 이 항이 매우 작아진다. 반면 순환 에지의 가중치 값이 |w| > 1이면 t-k가 클 때 $w^{t-k}$가 매우 커진다. t-k 값이 크다는 것은 긴 시간의 의존성을 가진다는 의미이다. 그레이디언트 소실이나 폭주를 피하는 단순한 방법은 |w|=1이 되도록 만드는 것이다.

이 문제를 해결하는 방법

* 그레이디언트 클리핑
* TBPTT(Truncated BackPropagation Through Time)
* LSTM(Long Short-Term Memory)

그레이디언트 클리핑을 사용하면 그레이디언트의 임계 값을 지정하고 이 값을 넘어서는 경우 임계값을 그레이디언트 값으로 사용한다.

TBPTT는 정방향 계산 후 역전파될 수 있는 타임 스텝의 횟수를 제한한다. 예를 들어 시퀀스가 100개의 원소 또는 스텝을 가지더라도 가장 최근 20번의 타임 스텝만 역전파할 수 있다.

그레이디언트 클리핑과 TBPTT가 그레이디언트 폭주를 해결할 수 있지만 그레이디언트가 시간을 거슬러 적절하게 가중치가 업데이트될 수 있는 스텝을 제한한다.

LSTM은 메모리 셀을 사용해서 그레이디언트 소실과 폭주 문제를 극복하여 긴 시퀀스를 성공적으로 모델링할 수 있다.

### 16.2.5 LSTM 셀

* LSTM의 기본 구성 요소는 일반 RNN의 은닉층을 표현 또는 대체하는 메모리 셀이다.
* 그레이디언트 소실과 폭주 문제를 극복하기 위해 각 메모리 셀에 적절한 가중치 w=1을 유지하는 순환 에지가 있다. 이 순환 에지의 출력을 셀 상태라고 한다.


In [13]:
# LSTM 셀
Image(url='https://git.io/JLdVR', width=700)

이전 타임 스텝의 셀 상태 $C^{(t-1)}$은 어떤 가중치와도 직접 곱해지지 않고 변경되어 현재 타임 스텝의 셀 상태 $C^{(t)}$을 얻는다.

메모리 셀의 정보 흐름은 다음에 기술된 몇 개의 연산 유닛(또는 게이트)으로 제어된다. 

* $\odot$는 원소별 곱셈
* $\oplus$는 원소별 덧셈
* $x^{(t)}$은 타임 스텝 t에서 입력 데이터
* $h^{(t-1)}$은 타임 스텝 $t-1$에서 은닉 유닛의 출력

네 개의 상자는 시그모이드 함수나 하이퍼볼릭 탄젠트 활성화 함수와 일련의 가중치로 표시된다. 이 상자는 입력($h^{(t-1)}$과 $x^{(t)}$에 대해 행렬-벡터 곱셈을 수행한 후 선형 조합된다. 시그모이드 함수로 계산하는 유닛을 게이트라고 하며 $\odot$를 통해 출력된다.

LSTM 셀에는 삭제 게이트, 입력 게이트, 출력 게이트가 있다.

* 삭제 게이트($f_t$)
    * 메모리 셀이 무한정 성장하지 않도록 셀 상태를 다시 설정한다.
    * 삭제 데이트가 통과할 정보와 억제할 정보를 결정한다.
    * $f_t = \sigma(W_{xf}x^{(t)} + W_{hf}h^{(t-1)} + b_f)$
* 입력 게이트($i_t$)와 후보 값($\tilde{C}_t$)은 셀 상태를 업데이트하는 역할을 담당
    * $i_t = \sigma( W_{xi}x^{(t)} + W_{hi}h^{(t-1)} + b_i )$
    * $ \tilde{C}_t = \tanh ( W x^{(t)} + W h^{(t-1)} + b_c ) $
    * 타임 t 에서 셀 상태
        * $C^{(t)} = (C^{(t-1)} \odot f_t) \oplus (i_t \odot \tilde{C}_t )$
* 출력 게이트($O_t$)는 은닉 유닛의 출력 값을 업데이트한다.
    * $o_t = \sigma (W_{xo} x^{(t)} + W_{ho} h^{(t-1)} + b_o )$
        * 위를 가지고 현재 타임 스텝에서 은닉 유닛의 출력을 아래와 같이 계산
            * $h^{(t)} = o_t \odot \tanh( C^{(t)} )$

LSTM의 경우 텐터플로에 최적화된 래퍼 함수로 이미 모두 구현되어 있어 간단하고 효율적으로 LSTM 셀을 정의할 수 있다.

* 다른 고급 RNN 모델
    * LSTM은 의존성이 긴 시퀀스를 모델링하는 기본적인 방법을 제공한다.
        * LSTM에는 많은 변종이 있다.
    * GRU
        * LSTM보다 구조가 단순하므로 계산 효율성이 높다.
        * 폴리포닉 음악 모델링 같은 일부 작업의 성능은 LSTM과 견줄만하다.


